In [ ]:
from ultralytics import YOLO

model = YOLO('/home/messyas/ml/jetson/runs/detect/yolov11xtrain/weights/best.pt') 

results = model.val(data='seu_dataset.yaml', split='test')
print(results.metrics.box.map) 


In [1]:
from ultralytics import YOLO

model_path = '/home/messyas/ml/jetson/runs/detect/yolov11xtrain2/weights/best.pt'
data_yaml_path = '/home/messyas/ml/data/tccdata/yolo_dataset/data.yaml'

try:
    model = YOLO(model_path)
    results = model.val(data=data_yaml_path, split='test', save_json=False)
    map_50_95 = results.box.map
    map_50 = results.box.map50
    try:
        recall = results.results_dict['metrics/recall(B)']
    except KeyError:
        print("A chave 'metrics/recall(B)' não foi encontrada. Verifique as chaves disponíveis:")
        print(results.results_dict.keys())
        recall = None

    f1_score = None
    if recall is not None:
        try:
            precision = results.results_dict['metrics/precision(B)']
         
            if (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)
            else:
                f1_score = 0
        except KeyError:
            print("A chave 'metrics/precision(B)' não foi encontrada.")
            precision = None

    print("\n--- Métricas YOLOv11x ---")
    print(f"mAP (50:95): {map_50_95:.4f} ({map_50_95 * 100:.2f} %)")
    print(f"mAP50:       {map_50:.4f} ({map_50 * 100:.2f} %)")

    if recall is not None:
        print(f"Revocação:   {recall:.4f} ({recall * 100:.2f} %)")
    else:
        print("Revocação:   N/A")

    if f1_score is not None:
        print(f"F1-Score:    {f1_score:.4f} ({f1_score * 100:.2f} %)")
    else:
        print("F1-Score:    N/A")

    yolov11x_data = {
        "Modelo": "YOLOv11x", 
        "mAP (%)": map_50_95 * 100,
        "mAP50 (%)": map_50 * 100,
        "Revocação (%)": recall * 100 if recall is not None else 'N/A',
        "F1-Score (%)": f1_score * 100 if f1_score is not None else 'N/A'
    }

    print("\nDados para YOLOv11x:")
    print(yolov11x_data)

except FileNotFoundError:
    print(f"Erro: O arquivo do modelo '{model_path}' ou o dataset '{data_yaml_path}' não foi encontrado.")
    print("Por favor, verifique os caminhos.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

KeyboardInterrupt: 